# Question & Answer System
by ES

import:

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import sys
import json 
import re
import gensim
import string
import math
import nltk
import random
import csv
import io
import spacy
import scipy
import heapq
from gensim.summarization import bm25
from collections import Counter
from nltk import pos_tag, word_tokenize
from nltk.tree import Tree
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords

C:\Users\ES\Anaconda3\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Open and load files:

In [2]:
with open('training.json','r') as f:
    training = json.load(f)

with open('documents.json','r') as f:
    documents = json.load(f)

with open('devel.json','r') as f:
    develop = json.load(f)
    
with open('testing.json','r') as f:
    testing = json.load(f)

TEXT SIMILARITY
Use bm25 to calculate the similarity between question and each paragraph in the target document. Get the paragraph with highest score.
For test set, return the question and paragraph.
For training/develop set, return the index of paragraph, index of right paragraph, and right answer as well

In [3]:
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

def wordPreprocess(word_list):
    stopWords = set(stopwords.words('english'))
    word_list_preprocessed = []
    for word in word_list:
        word_processed = lemmatize(word.lower())
        if word_processed not in stopWords and word_processed not in string.punctuation:
            word_list_preprocessed.append(word_processed)
    return word_list_preprocessed

documents_dict = {}
t = str(len(documents))
num = 0
for doc in documents:
    sys.stdout.write(('getting documents dict: '+'{0}/'+t+'\r').format(num + 1))
    sys.stdout.flush()
    para_list = []
    for para in doc["text"]:
        word_list = wordPreprocess(word_tokenize(para))
        para_list.append(word_list)
    documents_dict[doc["docid"]]=para_list
    num+=1

In [4]:
develop_data = []
t = str(len(develop))
num = 0
for question in develop:
    sys.stdout.write(('getting develop data: '+'{0}/'+t+'\r').format(num + 1))
    sys.stdout.flush()
    bm25Model = bm25.BM25(documents_dict[question["docid"]])
    average_idf = sum(map(lambda k: float(bm25Model.idf[k]), bm25Model.idf.keys())) / len(bm25Model.idf.keys())
    scores = bm25Model.get_scores(wordPreprocess(word_tokenize(question["question"])),average_idf)
    idx_list = heapq.nlargest(2, enumerate(scores), key=lambda x:x[1])
    idx, vals = zip(*idx_list)
    develop_data.append((question["question"],[documents[question["docid"]]["text"][idx[0]],documents[question["docid"]]["text"][idx[0]]],idx,question["answer_paragraph"],question["text"]))
    num+=1

In [5]:
test_data = []
t = str(len(testing))
num = 0
for question in testing:
    sys.stdout.write(('getting develop data: '+'{0}/'+t+'\r').format(num + 1))
    sys.stdout.flush()
    bm25Model = bm25.BM25(documents_dict[question["docid"]])
    average_idf = sum(map(lambda k: float(bm25Model.idf[k]), bm25Model.idf.keys())) / len(bm25Model.idf.keys())
    scores = bm25Model.get_scores(word_tokenize(question["question"]),average_idf)
    idx_list = heapq.nlargest(2, enumerate(scores), key=lambda x:x[1])
    idx, vals = zip(*idx_list)
    test_data.append((question["question"],[documents[question["docid"]]["text"][idx[0]],documents[question["docid"]]["text"][idx[0]]],idx))
    num+=1

Evaluate function:
print the acurate of similarity part

In [6]:
def tfidfAcurate(data_list):
    data_count = 0
    correct_count = 0
    for data in data_list:
        data_count+=1
        if data[2][0]==data[3] or data[2][1]==data[3]:
            correct_count +=1
    return float(correct_count)/data_count

print(tfidfAcurate(develop_data))

0.8737487891507911


Get Answer:

In [7]:
def getEntity(sentence):
    ## spacy
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(sentence)
    return doc.ents

def getTextByLabel(entitys,label_list):
    answer_list = []
    for lab in label_list:
        for entity in entitys:
            for ent in entity:
                label = ent.label_
                if label == lab:
                    answer_list.append(ent.text)
    return answer_list

def getAnswer(data_list):
    nlp = spacy.load('en_core_web_sm')
    t = str(len(data_list))
    now = 0
    answer=[]
#     print(type(answer))
    id_num = 0
#     count = [0,0,[0,0,0,0,0,0,0,0],[0,0,0,0],0,[0,0,0,0],0]
    useList = [0,0]
    for data in data_list:
        sys.stdout.write(('{0}/'+t+'\r').format(id_num + 1))
        sys.stdout.flush()
        paragraph_entity = [getEntity(data[1][0]),getEntity(data[1][1])]
        answer_list = list()
        
        if bool(re.search('who',data[0],re.IGNORECASE) or re.search('organization',data[0],re.IGNORECASE)):
#             count[0]+=1
            answer_list = getTextByLabel(paragraph_entity,["PERSON","NORP","ORG"])
        elif bool(re.search('when',data[0],re.IGNORECASE) or re.search('time',data[0],re.IGNORECASE) or re.search('month',data[0],re.IGNORECASE) or re.search('day',data[0],re.IGNORECASE) or re.search('year',data[0],re.IGNORECASE)):
#             count[1]+=1
            answer_list = getTextByLabel(paragraph_entity,["DATE","TIME","CARDINAL"])
        elif bool(re.search('where',data[0],re.IGNORECASE) or re.search('place',data[0],re.IGNORECASE) or re.search('city',data[0],re.IGNORECASE) or re.search('country',data[0],re.IGNORECASE)):
#             count[1]+=1
            answer_list = getTextByLabel(paragraph_entity,["GPE","LOC","FACILITY","ORG"])
        elif bool(re.search('how much',data[0],re.IGNORECASE) or re.search('how many',data[0],re.IGNORECASE)):
#             count[1]+=1
            answer_list = getTextByLabel(paragraph_entity,["PERCENT","QUANTITY","CARDINAL","MONEY"])
        
#         if bool(re.search('who',data[0],re.IGNORECASE) or re.search('name',data[0],re.IGNORECASE)):
#             count[0]+=1
#             answer_list = getTextByLabel(paragraph_entity,["PERSON","NORP","ORG"])
#         elif bool(re.search('where',data[0],re.IGNORECASE) or re.search('place',data[0],re.IGNORECASE)):
#             count[1]+=1
#             answer_list = getTextByLabel(paragraph_entity,["GPE","LOC","FACILITY","ORG"])
#         elif bool(re.search('what',data[0],re.IGNORECASE)):
#             if bool(re.search('what time',data[0],re.IGNORECASE)):
#                 count[2][0]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["DATE","TIME"])
#             elif bool(re.search('what year',data[0],re.IGNORECASE) or re.search('what day',data[0],re.IGNORECASE) or re.search('what date',data[0],re.IGNORECASE)):
#                 count[2][1]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["DATE"])
#             elif bool(re.search('what city',data[0],re.IGNORECASE) or re.search('what country',data[0],re.IGNORECASE)):
#                 count[2][2]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["GPE"])
#             elif bool(re.search('what value',data[0],re.IGNORECASE)):
#                 count[2][3]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["QUANTITY","MONEY","CARDINAL","PERCENT"])
#             elif bool(re.search('what percentage',data[0],re.IGNORECASE)):
#                 count[2][4]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["PERCENT"])
#             elif bool(re.search('event',data[0],re.IGNORECASE)):
#                 count[2][5]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["EVENT"])
#             elif bool(re.search('language',data[0],re.IGNORECASE)):
#                 count[2][6]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["LANGUAGE"])
#             else:
#                 count[2][7]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["PRODUCT","WORK_OF_ART","EVENT","FACILITY","ORG"])
#         elif bool(re.search('how',data[0],re.IGNORECASE)):
#             if bool(re.search('how much',data[0],re.IGNORECASE)):
#                 count[3][0]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["MONEY"])
#             if bool(re.search('how many',data[0],re.IGNORECASE)):
#                 count[3][1]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["QUANTITY","MONEY","CARDINAL","PERCENT"])
#             if bool(re.search('how does',data[0],re.IGNORECASE) or re.search('how was',data[0],re.IGNORECASE)):
#                 count[3][2]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["LAW","EVENT"])
#             else:
#                 count[3][3]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["QUANTITY","MONEY","CARDINAL"])
#         elif bool(re.search('when',data[0],re.IGNORECASE)):
#             count[4]+=1
#             answer_list = getTextByLabel(paragraph_entity,["DATE","TIME"])
#         elif bool(re.search('which',data[0],re.IGNORECASE)):
#             if bool(re.search('which person',data[0],re.IGNORECASE)):
#                 count[5][0]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["PERSON"])
#             elif bool(re.search('which time',data[0],re.IGNORECASE)):
#                 count[5][1]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["DATE","TIME"])
#             elif bool(re.search('which country',data[0],re.IGNORECASE)):
#                 count[5][2]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["NORP","ORG"])
#             else:
#                 count[5][3]+=1
#                 answer_list = getTextByLabel(paragraph_entity,["FACILITY","PRODUCT","WORK_OF_ART","EVENT","ORG"])
        else:
#             count[6]+=1
        for ans in answer_list:
            if ans in data[0]:
                answer_list.remove(ans)
                
        if len(answer_list)==0:
            useList[1]+=1
            
            try:
                if not len(paragraph_entity[0])==0:
                    for ent in paragraph_entity[0]:
                        if ent.text in data[0]:
                            paragraph_entity[0].remove(ent)
                    index = random.randint(0,len(paragraph_entity[0])-1) 
                    answer.append (([str(id_num)+','+paragraph_entity[0][index].text.replace(',','')],data[2]))
                else:
                    for ent in paragraph_entity[1]:
                        if ent.text in data[0]:
                            paragraph_entity[1].remove(ent)
                    index = random.randint(0,len(paragraph_entity[1])-1) 
                    answer.append (([str(id_num)+','+paragraph_entity[1][index].text.replace(',','')],data[2]))
            except:
                doc = nlp(data[1][0])
                index = random.randint(0,len(doc)-1)
                answer.append(([str(id_num)+','+doc[index].text.replace(',','')],data[2]))
        else:
            try:
                if answer_list [0] not in data[0]:
#                     print(answer_list[0])
                    answer.append(([str(id_num)+','+answer_list[0].replace(',','')],data[2]))
                elif answer_list [1] not in data[0]:
                    answer.append(([str(id_num)+','+answer_list[1].replace(',','')],data[2]))
                elif answer_list [2] not in data[0]:
                    answer.append(([str(id_num)+','+answer_list[2].replace(',','')],data[2]))
                else:
                    answer.append(([str(id_num)+','+answer_list[3].replace(',','')],data[2]))
            except:
                doc = nlp(data[1][0])
                index = random.randint(0,len(doc)-1)
                answer.append(([str(id_num)+','+doc[index].text.replace(',','')],data[2]))
            useList[0]+=1
        id_num +=1
#     print("count ",count)
    print("use list ",useList)
    return answer

In [8]:
# answer = getAnswer(test_data)
answer = getAnswer(develop_data)

count 097 [502, 150, [2, 158, 56, 0, 42, 15, 76, 1346], [42, 192, 10, 309], 147, [2, 1, 5, 245], 31]
use list  [2473, 624]


Evaluate function:
print the acurate of answer:

In [9]:
def campare(question,answer):
    if question[4].lower()==str(answer[0]).strip("'").split(',',1)[1][:-2].lower():
        return True
    else:
        print(question[0],"answer: "+question[4],"your answer: "+str(answer[0]).strip("'").split(',',1)[1][:-2])
        return False

def answerAcurate(data_list,answer):
    data_count = 0
    correct_count = 0
    results = map(campare,data_list,answer)
    for result in results:
        data_count+=1
        if result:
            correct_count +=1
    return float(correct_count)/data_count

print(answerAcurate(develop_data,answer))
    

On what date did the companies that became the Computing-Tabulating-Recording Company get consolidated? answer: june 16 , 1911 your answer: June 16 1911
What percentage of its desktop PCs does IBM plan to install Open Client on to? answer: 5 % your answer: 5%
What year did IBM hire its first black salesman? answer: 1946 your answer: 1935
IBM made an acquisition in 2009, name it. answer: spss your answer: Watson
This IBM invention is known by the acronym UPC, what is the full name? answer: universal product code your answer: Watson
In 2012 Fortune ranked the largest US firms by number employees, what was IBMs rank? answer: second largest your answer: Fortune) №1
How many gallons of liquid cleaning agent leaked from an IBM facility in 1979? answer: 4,100 gallons your answer: 4100 gallons
In what year did IBM get its name? answer: 1924 your answer: Virginia Rometty
DeveloperWorks has content about open industry standard technologies like Java and SOA, what is one other industry standard t

What enrichment process was used by the Manhattan Project? answer: gaseous diffusion your answer: UF\n
How much economically viable uranium is there in ore reserves, in millions of tonnes? answer: 5.5 your answer: 5.5 million tonnes
What was the first self-sustained nuclear chain reaction created by human beings called? answer: chicago pile-1 your answer: the Manhattan Project
What was the price of a kilogram of depleted uranium hexafluoride in 2001? answer: $ 5 your answer: 
What does a uranium(VI) cation form when it binds to two terminal oxides and three or more carbonates? answer: anionic complexes your answer: Pourbaix
In what city was uranium discovered? answer: berlin your answer: the United States
Who discovered the planet Uranus? answer: william herschel your answer: Martin Heinrich Klaproth
How many metric tons of uranium oxide was used in Chicago Pile-1? answer: 53 your answer: 400 short tons
What is 238U? answer: depleted uranium your answer: 
What is the natural abundance 

What is the sole fissile isotope that occurs in nature? answer: uranium-235 your answer: two
What country produced 5.5% of the world's concentrated uranium oxide in 2005? answer: uzbekistan your answer: Canada
What notable carbonates are often water soluble? answer: uranium your answer: Pourbaix
What type of alpha emitter is uranium? answer: weak your answer: similarly
What is another name for a fission/fusion bomb? answer: thermonuclear weapon your answer: plutonium-239
What is uranium used for most often in the military? answer: high-density penetrators your answer: Gulf War Syndrome
Along with uranyl sulfate and uranyl chloride, what compound is formed by the UO2+ 2 ion? answer: uranyl carbonate your answer: U(V
What happens to a majority of ingested uranium? answer: excreted your answer: up to 5%
What is a notable form uranium ore is converted into after extraction? answer: uranium dioxide your answer: Hydrochloric
At what level of exposure does uranium become imminently dangerous 

From what region in Portugal did immigrants come from to work in San Diego's fishing industry? answer: portuguese azores your answer: Little Italy
What is the population of San Diego's urgan area? answer: 2,956,746 your answer: 
In what year did San Diego officially become a city? answer: 1850 your answer: 1542
What cellular  company is one of the city's largest private-sector employers? answer: qualcomm your answer: Nokia LG Electronics
What park in San Diego hosted two World Fairs near the beginning of the 20th century? answer: balboa park your answer: the Panama-California Exposition
How many neophytes resided in the San Diego area in 1797? answer: 1,400 your answer: over 1400
After the land was claimed for Spain, how many years passed before the settlement of Alta began? answer: 200 your answer: the
What organization regulates building heights in San Diego? answer: federal aviation administration your answer: the El Cortez Hotel
Of the top 10 largest cities in the country, which pl

What was the name of Juan Rodriguez Cabrillo's ship? answer: san salvador your answer: Juan Rodríguez
How many bird species have been observed in San Diego? answer: 492 your answer: one
What institution has contributed to the growth of fuel biotechnology? answer: university of california your answer: the University of California San Diego
What flyway contributes to the diverse bird population in San Diego County? answer: pacific flyway your answer: the Audubon Society
What word describes San Diego County due to the many birds residing there? answer: birdiest your answer: the Audubon Society
Where is the next nearest commercial crossing at the border? answer: otay mesa your answer: Mexico
What areas experience the most significant temperature variations? answer: inland areas your answer: August
What birding event does the Audubon Society host every year? answer: christmas bird count your answer: and
In 2008, which racial group had the most individuals under the age of 18? answer: hispan

...

What was the Libyan coup called at first? answer: white revolution your answer: Philosophy of the Revolution
Where did Gaddafi travel to after he was kicked out of Sabha? answer: misrata your answer: Halyna Kolotnytska
Of what party did Gaddafi become president in this period? answer: arab socialist union your answer: the Yom Kippur War
What official name did the GNC give to Libya? answer: state of libya your answer: Gaddafi
By what name was Muammar Muhammad Abu Minyar al-Gaddafi typically known? answer: colonel gaddafi your answer: القذافي\u200e
Along with Lamin Khalifah Fhimah, who was suspected of the attack on Pan Am Flight 103? answer: abdelbaset al-megrahi your answer: Gaddafi
What was Libya's main export? answer: crude oil your answer: OPEC
In what month did the UN Security Council set up a no fly zone? answer: march your answer: the United Nations Security Council
What sort of identity was promoted by the RCC? answer: pan-libyan your answer: government
Who stated that Gaddafi w

What was the code name for Gaddafi's coup against the monarchy? answer: operation jerusalem your answer: Idris
As a result of the 1981 laws putting the government in control of commerce, what began to thrive? answer: black market your answer: GPC
What language did the Berbers living in the Nafusa Mountains not speak? answer: arabic your answer: country
What type of education did Gaddafi prefer to primary schools? answer: home schooling your answer: Sirte
Why did Libyan Arab Airlines Flight 114 accidentally fly into Israel's airspace? answer: sandstorm your answer: had
After Tripoli was conquered, where did Gaddafi go? answer: sirte your answer: Munich
What was the nationality of a significant number of teachers in Sabha? answer: egyptian your answer: Philosophy of the Revolution
Whose efforts did Gaddafi see himself as continuing? answer: nasser your answer: GPC
What was another name for the Great Man-Made River? answer: gaddafi 's pet project your answer: Gaddafi
What political philos

How close to the line of scrimmage can defensive players approach before a play is run? answer: one yard your answer: nearer than 24 yards
How long is a quarter in minutes? answer: 15 your answer: one
On which date was the first Canadian football game for which a written record has survived? answer: october 15 , 1862 your answer: the Second Battalion Scots Fusilier Guards
Which round of tie-breaking is the last possible round in a regular season CFL game? answer: second your answer: to
On which date was the first governing body for Canadian football formed? answer: march 24 , 1873 your answer: Rugby
How many players must the team with possession place near the line of scrimmage? answer: seven your answer: 1
In which year did the CFL begin requiring teams to attempt 2-point conversions after scoring in a tie-breaking situation? answer: 2010 your answer: first
Where does the next play start unless a team has just scored? answer: scrimmage your answer: (
How many years did the CFL South D

What was the first Dutch Bible translation called? answer: statenvertaling your answer: the Burgundian Ducal Court
How would you tell a West Flemish speaker "yes"? answer: ja your answer: on
What word was used in the Oaths of Strasbourg for the oath's Germanic section? answer: teudisca your answer: the Old Germanic
What percentage of people in Belgium speak Dutch? answer: 59 % your answer: 96%
It took until the end of what century for people in French cities like Gravelines to finally switch over to French from Dutch? answer: 19th your answer: World War I
What West Central German dialect of Dutch contains the name of a U.S. state? answer: pennsylvania dutch your answer: West Germanic
What type of clause has the same word order in Dutch as a tag question? answer: declarative clause your answer: 's
Towards the end of what century did it become important for more people in Dutch colonies to speak Dutch? answer: 19th your answer: Bible
What word do Dutch speakers use for what we call a "sh

What poet wrote a notable rhyme about Laemmle? answer: ogden nash your answer: Academy Award
What was the studio that Alfred Hitchcock normally worked for? answer: selznick international pictures your answer: The Music Corporation of America
By what month and year did Laemmle change his opinion on "unclean pictures"? answer: april 1927 your answer: Universal
Prior to his work in America, where was Joe Pasternak employed? answer: universal 's german subsidiary your answer: Universal
When did Universal inaugurate its studio tour subsidiary? answer: 1964 your answer: mid-1962
In what year did Universal-International take over Castle Films? answer: 1951 your answer: 1946
What was the nationality of Matsushita Electric? answer: japanese your answer: MCA
Who partnered with Universal in the creation of Cinema International Corporation? answer: paramount pictures your answer: E.T.
In what state was the Universal Film Manufacturing Company incorporated? answer: new york your answer: The Univers

Write function:

In [10]:
##Write into csv file
def writeFile(answer):
#     sys.stdout = io.TextIOWrapper(sys.stdout.buffer,encoding='utf8')
    out = open('answer.csv','a',newline='')
    csv_write =csv.writer(out,dialect='excel')
    csv_write.writerow(["id,answer"])
    answerID = 0
    for line in answer:
        try:
            csv_write.writerow(line[0])
        except:
            print("line "+str(answerID)+" cannot write： "+str(line[0]))
        answerID+=1
    out.close()
    
writeFile(answer)
